In [1]:
import requests
import time
import pprint
import pickle
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
import xml.etree.ElementTree as ET
import re
from unicodedata import normalize
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot
import seaborn as sns
%matplotlib
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 10)

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
def parseo_cuatrimeste_fecha(row):
    if row == '01/01/':
        date = 'Primero'
    elif row == '01/04/':
        date = 'Segundo'
    elif row == '01/07/':
        date = 'Tercero'
    elif row == '01/10/':
        date = 'Cuarto'
    return date

In [3]:
data = pd.read_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Modelo evolución precios\Datos - copia - Marta\tabla.csv', 
                   sep=';', engine="python", decimal=',', encoding="latin1")
data= data.dropna()
data= data.loc[:,["Fecha", "Provincia", "IMPTOTAL", "SUPERF_ADOPTADA", "MATRICULA", "MES", "AÑO"]]
data.columns= ["Fecha", "Provincia_x", "IMPTOTAL", "SUPADAPT", "MATRICULA", "MES", "AÑO"]

In [4]:
data = data[data['SUPADAPT']>1]
data['Precio m2'] = data.loc[:, ['IMPTOTAL', 'SUPADAPT']].apply(lambda s: s[0]/s[1], axis=1)
data = data[(data['Precio m2']>300)&(data['Precio m2']<36000)]

In [5]:
data

,Fecha,Provincia_x,IMPTOTAL,SUPADAPT,MATRICULA,MES,AÑO,Precio m2
0,2018-07-01,48.0,159414.31,88.99,719000001124,SEP,2018,1791.373300
1,2018-10-01,20.0,613418.67,247.13,719000002322,NOV,2018,2482.169992
2,2018-07-01,8.0,514570.56,150.93,719000006953,JUL,2018,3409.332538
3,2017-10-01,36.0,70629.75,75.00,719000008350,OCT,2017,941.730000
4,2018-04-01,12.0,100071.36,68.50,719000008930,MAY,2018,1460.895766
...,...,...,...,...,...,...,...,...
84730,2018-10-01,11.0,201674.49,181.30,719237693052,OCT,2018,1112.379978
84731,2018-07-01,37.0,123657.20,164.00,719237718321,JUL,2018,754.007317
84732,2018-04-01,17.0,143682.79,152.10,719237737241,JUN,2018,944.660026
84733,2018-04-01,8.0,169543.08,171.00,719237759916,JUN,2018,991.480000


In [6]:
trimestre = data.loc[data.Fecha == '2018-10-01', ['Provincia_x', 'Precio m2']]  # insertar último cuatrimestre de la tabla
#De cada vivienda tenemos el precio del m2 a fecha 01/10/2018

#Calcular precio medio del último cuatrimestre por provincia a fecha de 01/10/2018
trimestre= trimestre.sort_values('Provincia_x')
trimestre['Media_ULT_CUAT'] =trimestre.groupby('Provincia_x')['Precio m2'].transform('mean')
trimestre = trimestre.loc[:,['Provincia_x', 'Media_ULT_CUAT']]
trimestre = trimestre.drop_duplicates()
trimestre

,Provincia_x,Media_ULT_CUAT
83852,1.0,1432.074103
21355,2.0,1221.841535
78721,3.0,1164.967177
82716,4.0,1094.620093
78036,5.0,647.350038
83176,6.0,736.695715
84011,7.0,2468.773754
83410,8.0,2210.529337
32706,9.0,1178.628354
7894,10.0,966.008001


In [7]:
tablin_value_updated = data.merge(trimestre, left_on=['Provincia_x'], right_on=['Provincia_x'])
tablin_value_updated= tablin_value_updated.drop_duplicates()
tablin_value_updated.columns= ['Fecha', 'Provincia', 'IMPTOTAL', 'SUPADAPT', 'MATRICULA','MES', 'Año', 'Precio m2', 'Media_ULT_CUAT']
data.columns= ['Fecha', 'Provincia', 'IMPTOTAL', 'SUPADAPT', 'MATRICULA','MES', 'AÑO', 'Precio m2']

#De cada vivienda tenemos el precio del m2 a fecha 01/10/2018

In [8]:
año= data
año['Año'] = año.Fecha.apply(lambda s: s[6:])
#Calcular precio medio del último año (2018) por provincia 
año= año.sort_values('Provincia')
año['Media_ULT_AÑO'] =año.groupby('Provincia')['Precio m2'].transform('mean')
año = año.loc[:,['Provincia', 'Media_ULT_AÑO']]
año= año.drop_duplicates()
año

,Provincia,Media_ULT_AÑO
25728,1.0,1596.647761
24149,2.0,997.343589
12767,3.0,1122.180168
4081,4.0,1012.976355
5074,5.0,870.012827
9795,6.0,930.202556
16022,7.0,2209.984867
38945,8.0,2038.569993
39374,9.0,1108.835092
68053,10.0,952.646398


In [9]:
tablin_value_updated = tablin_value_updated.merge(año, left_on=['Provincia'], right_on=['Provincia'])
tablin_value_updated= tablin_value_updated.drop_duplicates()
tablin_value_updated = tablin_value_updated.loc[:,['MATRICULA', 'Provincia', 'Fecha', 'IMPTOTAL', 'SUPADAPT', 'Precio m2', 'Media_ULT_CUAT','Media_ULT_AÑO', 'Año', 'MES']]


In [10]:
trimestre = data.loc[(data['MES'] == 'JUN')& (data['AÑO'] == 2018), ['Provincia', 'Precio m2']]
# insertar último trimestre de la tabla
#De cada vivienda tenemos el precio del m2 a fecha JUN 2018

#Calcular precio medio del último trimestre por provincia a fecha de JUN 2018
trimestre= trimestre.sort_values('Provincia')
trimestre['Media_ULT_TRIM'] =trimestre.groupby('Provincia')['Precio m2'].transform('mean')
trimestre = trimestre.loc[:,['Provincia', 'Media_ULT_TRIM']]
trimestre = trimestre.drop_duplicates()

In [11]:
tablin_value_updated = tablin_value_updated.merge(trimestre, left_on=['Provincia'], right_on=['Provincia'])
tablin_value_updated= tablin_value_updated.drop_duplicates()
tablin_value_updated = tablin_value_updated.loc[:,['MATRICULA', 'Provincia', 'Fecha', 'IMPTOTAL', 'SUPADAPT', 'Precio m2', 'Media_ULT_CUAT','Media_ULT_AÑO','Media_ULT_TRIM', 'Año', 'MES']]


In [12]:
tablin_value_updated['Variacion_CUAT'] = (tablin_value_updated["Precio m2"] - tablin_value_updated["Media_ULT_CUAT"])/tablin_value_updated["Precio m2"]
tablin_value_updated['Variacion_AÑO'] = (tablin_value_updated["Precio m2"] - tablin_value_updated["Media_ULT_AÑO"])/tablin_value_updated["Precio m2"]
tablin_value_updated['Variacion_TRIM'] = (tablin_value_updated["Precio m2"] - tablin_value_updated["Media_ULT_TRIM"])/tablin_value_updated["Precio m2"]

In [13]:
tablin_value_updated = tablin_value_updated.loc[:,['MATRICULA','Provincia','Fecha','Media_ULT_CUAT', 'Media_ULT_AÑO', 'Variacion_CUAT','Variacion_AÑO', 'Variacion_TRIM', 'Año', 'MES']]
tablin_value_updated    #variación de cada vivienda respecto al último cuatrimestre o al último año

,MATRICULA,Provincia,Fecha,Media_ULT_CUAT,Media_ULT_AÑO,Variacion_CUAT,Variacion_AÑO,Variacion_TRIM,Año,MES
0,719000001124,48.0,2018-07-01,1996.070847,2135.838691,-0.114269,-0.192291,-0.327271,2018,SEP
1,719002482600,48.0,2017-10-01,1996.070847,2135.838691,0.100229,0.037225,-0.071771,2017,NOV
2,719002934245,48.0,2015-04-01,1996.070847,2135.838691,0.045604,-0.021224,-0.136837,2015,APR
3,719005347536,48.0,2018-07-01,1996.070847,2135.838691,-0.224583,-0.310330,-0.458673,2018,JUL
4,719005599648,48.0,2015-04-01,1996.070847,2135.838691,0.325547,0.278321,0.196619,2015,JUN
...,...,...,...,...,...,...,...,...,...,...
84146,719228514003,52.0,2018-04-01,1769.405951,1565.234850,-0.119984,0.009251,-0.009836,2018,JUN
84147,719228847945,52.0,2018-07-01,1769.405951,1565.234850,-0.396391,-0.235262,-0.259060,2018,JUL
84148,719229098739,52.0,2018-07-01,1769.405951,1565.234850,-0.034886,0.084529,0.066892,2018,JUL
84149,719231167938,52.0,2018-10-01,1769.405951,1565.234850,0.131418,0.231643,0.216840,2018,NOV


In [14]:
tablin_value_updated= tablin_value_updated.sort_values(by=['Provincia'])
#Calcular la media del precio de las viviendas por provincia
tablin_value_updated['Media_VAR_CUAT'] =tablin_value_updated.groupby(['Provincia','Fecha'])['Variacion_CUAT'].transform('mean')
tablin_value_updated['Media_VAR_TRIM'] =tablin_value_updated.groupby(['Provincia','Fecha'])['Variacion_TRIM'].transform('mean')

tablin_value_updated['Media_VAR_AÑO'] =tablin_value_updated.groupby(['Provincia', 'Fecha'])['Variacion_AÑO'].transform('mean')
tablin_value_updated = tablin_value_updated.loc[:,['Provincia','Fecha','Media_ULT_CUAT', 'Media_ULT_AÑO', 'Media_VAR_CUAT','Media_VAR_AÑO', 'Media_VAR_TRIM',  'Año', 'MES']]
tablin_value_updated= tablin_value_updated.drop_duplicates()     
#variación cuatrimestral y anual de cada provincia
#Esta tabla describe la variación de precios por provincia del último año o del último cuatrimestre respecto a la fecha 

In [15]:
ultaño= tablin_value_updated.loc[tablin_value_updated.Fecha == '2018-01-01', ['Provincia','Media_VAR_AÑO']]  # insertar último cuatrimestre de la tabla
#Variación último año

In [16]:
seis= tablin_value_updated[(tablin_value_updated['MES']=='JUN')]
seis= seis[(seis['Año']==2018)]
seis= seis.loc[:,['Provincia','Media_VAR_TRIM']]
#Variación últimos 6 meses

In [17]:
cod = pd.read_excel(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\codprov.xls', skiprows=1)
keys = cod['CODIGO']
values = cod['LITERAL']
dictionary = dict(zip(keys, values))
seis['Provincia'] = seis['Provincia'].map(dictionary)
ultaño['Provincia'] = ultaño['Provincia'].map(dictionary)

In [23]:
ultaño= ultaño.drop_duplicates()

In [33]:
variacion= ultaño.merge(seis, how='outer')
variacion.columns= ['Provincia', 'AÑO', '6MESES']

In [35]:
variacion.to_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\variacion_precios.csv', 
                   sep=';', encoding="latin1", decimal=',', index=False)

In [37]:
variacion

,Provincia,AÑO,6MESES
0,Araba/Álava,-0.145614,-0.256394
1,Albacete,-0.026450,-0.172834
2,Alicante/Alacant,-0.199681,-0.242358
3,Almería,-0.012074,-0.190576
4,Ávila,-0.213974,-0.123935
5,Badajoz,-0.450919,-0.166504
6,"Balears, Illes",-0.055506,-0.178434
7,Barcelona,-0.157094,-0.193804
8,Burgos,-0.179802,-0.297905
9,Cáceres,0.100488,-0.253439
